In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from time import sleep
from datetime import datetime
from splinter import Browser
from selenium.webdriver.common.keys import Keys
import time
import json
import os
import shutil

In [2]:
# Path to chromedriver.exe
executable_path = {'executable_path':'C:/Users/zacha/AppData/Local/bin/chromedriver.exe'}

# Open the Splinter Browswer
browser = Browser('chrome', **executable_path)

In [3]:
# URL to find information on Coronavirus
url1 = "https://txdshs.maps.arcgis.com/apps/opsdashboard/index.html#/ed483ecd702b4298ab01e8b9cafc8b83"

url2 = "https://dshs.texas.gov/coronavirus/"

url3 = "http://publichealth.harriscountytx.gov/Resources/2019-Novel-Coronavirus/Harris-County-COVID-19-Confirmed-Cases"

url4 = "https://www.khou.com/section/news/health/coronavirus"

In [4]:
# Visiting the url 
browser.visit(url1)

In [5]:
time.sleep(7)# Creating the Beautiful soup for URL
html = browser.html
soup = bs(html, 'html.parser')

In [6]:
# All the Texas counties
# counties = soup.find('nav', attrs={'class':'feature-list'})

# getting all the spans with the information for each Texas county
# rows = counties.find_all('span')




In [7]:
# counter = 0

# county = []
# cases = []

# for row in rows:
#     try:
#         print(rows[counter].text.strip().split('\n'))
#         county.append(rows[counter].text.strip().split('\n')[0])
#         cases.append(rows[counter].text.strip().split('\n')[1])
#         counter = counter + 3
        
#     except:
#         print('failure')

In [8]:
# df = pd.DataFrame(list(zip(county, cases)), columns =['Texas_County', 'COVID_19_Cases'])
# df

In [9]:
# Droping the word 'County'
# df['Texas_County'] = df['Texas_County'].apply(lambda x : x.split(' County')[0])

In [10]:
# Revise? 
reported_cases = soup.find_all('text')[6].text
reported_cases

'5,330'

In [11]:
deaths = soup.find_all('text')[8].text
deaths

'90'

In [12]:
df_stats = pd.DataFrame({'Cases':[reported_cases], 'Deaths':[deaths]})
df_stats.to_csv('static/csv/stats.csv', index=False)

In [13]:
# Getting Total Cases and Deaths 
browser.visit(url2)

In [14]:
browser.find_by_text('Accessible version').click()
time.sleep(5)

In [15]:
source = 'C:/Users/zacha/Downloads/'
destination = 'C:/Users/zacha/Documents/COVID-19/'
filename = 'CaseCountData.xlsx'

In [16]:
shutil.move(os.path.join(source, filename), os.path.join(destination, filename))

'C:/Users/zacha/Documents/COVID-19/CaseCountData.xlsx'

In [17]:
# dest = shutil.move(source, destination, )

In [18]:
df = pd.read_excel('CaseCountData.xlsx', header=1).dropna()
df.sort_values(by=['Positive'], ascending=False, inplace=True)
df['County'] = df.apply(lambda x: x['County'].rstrip(), axis=1)
df

,No.,County,Positive,Fatalities
57,58.0,Harris,847,6
29,30.0,Dallas,731,15
125,126.0,Travis,305,3
121,122.0,Tarrant,288,3
32,33.0,Denton,231,4
...,...,...,...,...
115,116.0,San Jacinto,1,0
49,50.0,Gonzales,1,0
48,49.0,Goliad,1,0
47,48.0,Gillespie,1,0


In [19]:
browser.visit(url3)

In [20]:
harris_table = browser.find_by_id('covidtable')

In [21]:
html_harris = browser.html
harris_soup = bs(html_harris, 'html.parser')

In [22]:
harris_data = []
table = harris_soup.find('table', attrs={'id':'covidtable'})
table_body = table.find('tbody')

rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    harris_data.append([ele for ele in cols if ele]) # Get rid of empty values

In [23]:
columns = ['Patient', 'Sex', 'Age_Range', 'Location','Exposure', 'Status']


In [24]:
harris_data = harris_data[1:]

In [25]:
harris_df = pd.DataFrame(harris_data, columns=columns)

In [26]:
harris_df.sort_values(by=['Age_Range'], inplace=True)

In [27]:
harris_df

,Patient,Sex,Age_Range,Location,Exposure,Status
43,44,Male,0 - 9,NW,Possible Exposure to Case,Active
45,46,Female,0 - 9,NW,Possible Exposure to Case,Active
172,173,Female,0 - 9,NW,Possible Exposure to Case,Active
185,186,Female,0 - 9,SE,Possible Exposure to Case,Active
147,148,Male,10 - 19,SE,Possible Exposure to Case,Active
...,...,...,...,...,...,...
154,155,Female,80+,SE,Investigation Pending,Active
246,247,Female,80+,SW,Investigation Pending,Active
15,16,Male,80+,NW,No Known Exposure,Deceased
158,159,Female,80+,NW,History of Travel,Active


In [28]:
harris_df.to_csv('static/csv/harris_stats.csv', index=False, header=True)

In [29]:
harris_df.head()

,Patient,Sex,Age_Range,Location,Exposure,Status
43,44,Male,0 - 9,NW,Possible Exposure to Case,Active
45,46,Female,0 - 9,NW,Possible Exposure to Case,Active
172,173,Female,0 - 9,NW,Possible Exposure to Case,Active
185,186,Female,0 - 9,SE,Possible Exposure to Case,Active
147,148,Male,10 - 19,SE,Possible Exposure to Case,Active


In [30]:
All_Tex_Counties_df = pd.read_csv('static/csv/Texas_Counties.csv')
All_Tex_Counties_df.head()

,X (Lat),Y (Long),CNTY_NM,CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,-97.492799,29.456415,Gonzales,90,48177,2.124911,0.257805,"(-97.492799, 29.456415)"
1,-98.697292,27.043405,Jim Hogg,125,48247,2.271751,0.267624,"(-98.697292, 27.043405)"
2,-97.681378,26.924094,Kenedy,66,48261,5.067864,0.389397,"(-97.681378, 26.924094)"
3,-96.965687,30.310651,Lee,144,48287,2.213853,0.153990,"(-96.965687, 30.310651)"
4,-95.853568,32.211881,Henderson,108,48213,3.152909,0.235056,"(-95.853568, 32.211881)"


In [31]:
All_Tex_Counties = All_Tex_Counties_df['CNTY_NM'].tolist()

In [32]:
df_counties = df['County'].tolist()

In [33]:
Counties_with_Cases = []

for i in df_counties:
    
    if i.rstrip() in All_Tex_Counties:
        Counties_with_Cases.append(i.rstrip())
        print(f'{i}---found')
        
    elif i == 'DeWitt':
        Counties_with_Cases.append('De Witt')
        print(f'{i}---found')
    else:
        print(f'Houston we have a problem: {i}')

Harris---found
Dallas---found
Travis---found
Tarrant---found
Denton---found
Bexar---found
Fort Bend---found
Collin---found
Galveston---found
Montgomery---found
Brazoria---found
Lubbock---found
El Paso---found
Brazos---found
Hidalgo---found
Webb---found
Williamson---found
McLennan---found
Smith---found
Bell---found
Hays---found
Nueces---found
Wichita---found
Cameron---found
Jefferson---found
Matagorda---found
Ellis---found
Victoria---found
Guadalupe---found
Randall---found
Johnson---found
Washington---found
Midland---found
Potter---found
Taylor---found
Hardin---found
Ector---found
Comal---found
Grayson---found
Rockwall---found
Bastrop---found
Bowie---found
Chambers---found
Wharton---found
Tom Green---found
Kendall---found
Calhoun---found
Castro---found
Walker---found
Gregg---found
Hockley---found
Orange---found
Hood---found
Moore---found
Rusk---found
Wilson---found
Harrison---found
Val Verde---found
Angelina---found
Starr---found
Cherokee---found
Waller---found
Kaufman---found
Nacogdoch

In [34]:
All_Tex_Counties_df = All_Tex_Counties_df.set_index('CNTY_NM')

In [35]:
Geo_df = All_Tex_Counties_df.loc[Counties_with_Cases]
Geo_df.head()

,X (Lat),Y (Long),CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
CNTY_NM,,,,,,,
Harris,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
Dallas,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
Travis,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
Tarrant,-97.291165,32.771852,220,48439,1.905740,0.225294,"(-97.291165, 32.771852)"
Denton,-97.116812,33.205574,61,48121,1.969324,0.238359,"(-97.116812, 33.205574)"


In [36]:
Geo_df = Geo_df.reset_index()
Geo_df = Geo_df.rename(columns={"CNTY_NM": "County"})
Geo_df

,County,X (Lat),Y (Long),CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,Harris,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
1,Dallas,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
2,Travis,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
3,Tarrant,-97.291165,32.771852,220,48439,1.905740,0.225294,"(-97.291165, 32.771852)"
4,Denton,-97.116812,33.205574,61,48121,1.969324,0.238359,"(-97.116812, 33.205574)"
...,...,...,...,...,...,...,...,...
138,San Jacinto,-95.166833,30.579489,204,48407,2.203357,0.153019,"(-95.166833, 30.579489)"
139,Gonzales,-97.492799,29.456415,90,48177,2.124911,0.257805,"(-97.492799, 29.456415)"
140,Goliad,-97.426461,28.657086,89,48175,2.171169,0.205451,"(-97.426461, 28.657086)"
141,Gillespie,-98.946272,30.318002,87,48171,2.150997,0.257844,"(-98.946272, 30.318002)"


In [37]:
result = pd.merge(df, Geo_df, on='County')
result = result.rename(columns={'X (Lat)':'long', 'Y (Long)':'lat'})

result.head()

,No.,County,Positive,Fatalities,long,lat,CNTY_NBR,FIPS,Shape_Leng,Shape_Area,County Location
0,58.0,Harris,847,6,-95.397821,29.859671,102,48201,7.731110,0.422798,"(-95.397821, 29.859671)"
1,30.0,Dallas,731,15,-96.777819,32.766537,57,48113,1.907782,0.226421,"(-96.777819, 32.766537)"
2,126.0,Travis,305,3,-97.781947,30.334233,227,48453,2.458272,0.248996,"(-97.781947, 30.334233)"
3,122.0,Tarrant,288,3,-97.291165,32.771852,220,48439,1.905740,0.225294,"(-97.291165, 32.771852)"
4,33.0,Denton,231,4,-97.116812,33.205574,61,48121,1.969324,0.238359,"(-97.116812, 33.205574)"


In [38]:
result = result[['County', 'Positive', 'Fatalities', 'lat', 'long']]
result

,County,Positive,Fatalities,lat,long
0,Harris,847,6,29.859671,-95.397821
1,Dallas,731,15,32.766537,-96.777819
2,Travis,305,3,30.334233,-97.781947
3,Tarrant,288,3,32.771852,-97.291165
4,Denton,231,4,33.205574,-97.116812
...,...,...,...,...,...
137,San Jacinto,1,0,30.579489,-95.166833
138,Gonzales,1,0,29.456415,-97.492799
139,Goliad,1,0,28.657086,-97.426461
140,Gillespie,1,0,30.318002,-98.946272


In [39]:
def df_to_geojson(df, properties, lat='lat', lon='long'):
    prefix = 'var' + ' data' + ' = '
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point','coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lat],row[lon]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

cols = ['County', 'Positive', 'Fatalities']
javascript_file = df_to_geojson(result, cols)
# {'type': 'FeatureCollection', 'features': [{'geometry': {'type': 'Point', 'coordinates': [-70.5867, -29.9953, 760]}, 'type': 'Feature', 'properties': {'name': 'A', 'description': 'Place a'}}, {'geometry': {'type': 'Point', 'coordinates': [-70.4933, -30.1217, 1250]}, 'type': 'Feature', 'properties': {'name': 'B', 'description': 'Place b'}}, {'geometry': {'type': 'Point', 'coordinates': [-70.5008, -30.0953, 1185]}, 'type': 'Feature', 'properties': {'name': 'C', 'description': 'Place c'}}]}

In [40]:

with open('static/js/map.json','w') as f:
    json.dump(javascript_file, f, indent=4, sort_keys=False)

In [41]:
browser.visit(url4)

In [42]:
headlines_harris = browser.html
headline_soup = bs(headlines_harris, 'html.parser')

In [43]:
# lis = headline_soup.find_all('li', class_="headline-list-with-abstract__item")
links = headline_soup.find_all('div', class_="headline-list-with-abstract__headline")

titles = []
hrefs = []

In [44]:
recent = headline_soup.find_all('div', class_="story-snapshot-with-abstract__headline")[0]

In [45]:
recent

<div class="story-snapshot-with-abstract__headline">
<a class="story-snapshot-with-abstract__link" data-tracking="story-snapshot-with-abstract/headlineLinkClicked" href="/article/news/health/coronavirus/city-officials-provide-update-on-coronavirus-outbreak-at-nursing-home/285-d5a728d6-77ea-4ff5-b9c7-161c6bb3a2a8">
<div class="story-snapshot-with-abstract__desktop-title dot-ellipsis dot-height-180" style="overflow-wrap: break-word;">
                        67 residents at San Antonio nursing home test positive, 1 death
                    </div>
<div class="story-snapshot-with-abstract__mobile-title dot-ellipsis dot-height-135" style="overflow-wrap: break-word;">
                        67 residents at San Antonio nursing home test positive, 1 death
                    </div>
</a>
</div>

In [46]:
recent_title = recent.find('div', class_="story-snapshot-with-abstract__desktop-title dot-ellipsis dot-height-180").text.strip()
recent_title

'67 residents at San Antonio nursing home test positive, 1 death'

In [47]:
titles.append(recent_title)
hrefs.append('https://www.khou.com' + recent.a['href'])

In [48]:
links

[<div class="headline-list-with-abstract__headline">
 <a class="headline-list-with-abstract__link" data-tracking="headline-list-with-abstract/headlineLinkClicked" href="/article/news/health/coronavirus/coronavirus-updates-houston-live-blog-april-3/285-6d1d2743-5612-4773-8627-3b9622f845d8">
                     Coronavirus updates: ExxonMobil donating $250k to Houston, Montgomery County food banks
                 </a>
 </div>, <div class="headline-list-with-abstract__headline">
 <a class="headline-list-with-abstract__link" cnx-title="5" data-islink="true" href="">This Day In History
     </a>
 </div>, <div class="headline-list-with-abstract__headline">
 <a class="headline-list-with-abstract__link" data-tracking="headline-list-with-abstract/headlineLinkClicked" href="/article/news/verify/verify-truths-and-facts-about-chloroquine/285-e3cc8d0a-86c1-475e-a6e0-b2a8953353b2">
                     VERIFY: Fact vs. Fiction of chloroquine, Malaria drug claimed to help treat COVID- 19
          

In [49]:

for article in links:
    
    try:
        
        title = article.find('a', class_="headline-list-with-abstract__link").text
        
        link = article.a['href']
        
        if (title and link):
            
            
            print('-------------')
            print(title)
            print(link)
            titles.append(title.strip())
            hrefs.append('https://www.khou.com' + link)
            
    except:
        print('no')

-------------

                    Coronavirus updates: ExxonMobil donating $250k to Houston, Montgomery County food banks
                
/article/news/health/coronavirus/coronavirus-updates-houston-live-blog-april-3/285-6d1d2743-5612-4773-8627-3b9622f845d8
-------------

                    VERIFY: Fact vs. Fiction of chloroquine, Malaria drug claimed to help treat COVID- 19
                
/article/news/verify/verify-truths-and-facts-about-chloroquine/285-e3cc8d0a-86c1-475e-a6e0-b2a8953353b2
-------------

                    Update: Fourth person dies from COVID-19 in Brazos County, 68 confirmed cases
                
/article/news/brazos-county-reports-cases-of-coronavirus/285-bb82428a-417e-4287-b7f8-ef6cf60ffa2b
-------------

                    Tyson Foods giving $60 million ‘thank you’ bonuses to truckers and frontline workers
                
/article/news/health/coronavirus/tyson-foods-giving-60-million-thank-you-bonuses-to-truckers-and-frontline-workers/285-126dd638-28e2-

In [50]:
latest_headlines_df = pd.DataFrame(list(zip(titles, hrefs)), columns =['Headline', 'link'])

In [51]:
latest_headlines_df

,Headline,link
0,67 residents at San Antonio nursing home test ...,https://www.khou.com/article/news/health/coron...
1,Coronavirus updates: ExxonMobil donating $250k...,https://www.khou.com/article/news/health/coron...
2,"VERIFY: Fact vs. Fiction of chloroquine, Malar...",https://www.khou.com/article/news/verify/verif...
3,Update: Fourth person dies from COVID-19 in Br...,https://www.khou.com/article/news/brazos-count...
4,Tyson Foods giving $60 million ‘thank you’ bon...,https://www.khou.com/article/news/health/coron...
5,Stocks pull lower after wavering earlier; crud...,https://www.khou.com/article/news/health/coron...
6,"Need a job? These Houston-area stores, other b...",https://www.khou.com/article/news/health/coron...


In [52]:
latest_headlines_df.to_csv('static/csv/headlines.csv', index=False, header=True)

In [53]:
browser.quit()